Artificial neural networks (ANNs) consist of input, hidden, and output layers with connected neurons (nodes) to simulate the human brain

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.14.0'

# Data Processing

In [4]:
ds = pd.read_csv("dataset/Churn_Modelling.csv")
ds

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


## Set matrix variable x and y 

In [5]:
# it will take all rows and all columns starting from index 3 instead of the last column
X = ds.iloc[:,3:-1].values 

#it will take all rows and  last column
Y = ds.iloc[:,-1].values 

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(Y)

[1 0 1 ... 1 1 0]


# Part 1: Encoding categorical data

# Label Encoding the "Gender" column

In [8]:
# we are doing encoding as there is only two values, so we can encode them either 0 or 1

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


# One Hot Encoding the "Geography" column

In [9]:
# as this field has multiple values so we can just simply encode them as 0 or 1, 
# we need a special type of encoding which Hot Encoding

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Splitting the dataset into the Training set and Test set

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [12]:
#Feature scaling is the process of normalizing the range of features in a dataset. 
#Real-world datasets often contain features that are varying in degrees of magnitude, range, and units. 
#Therefore, in order for machine learning models to interpret these features on the same scale, 
#we need to perform feature scaling

In [13]:
# we need to do feature scaling for deep learning

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Building the ANN

# Initializing the ANN

In [17]:
ann = tf.keras.models.Sequential();

# Adding the input layer and the first hidden layer

In [18]:
# The activation function of a fully connected neuron netwrok is rectifier activation function
# this will create a shallow hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the second hidden layer

In [19]:
# we can add another hidden layer to create a deep hidden network for ths

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the output layer

In [20]:
# now we need to add the output layer, as here we are expecting the output as 0 or 1 so the dimention of this binary
# output will be 1, also we always use the activation function for output layer as sigmoid so that it can covert the 
# final result to a probability

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

# Compiling the ANN

In [22]:
# The results of the Adam optimizer are generally better than every other optimization algorithm, 
# have faster computation time, and require fewer parameters for tuning. Because of all that, 
# Adam is recommended as the default optimizer for most of the applications

# when we are calculating a binary outcome the loss function will be binary_crossentropy, 
# for non bianry it will be categorical_crossentropy and the activation function will be softmax



ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN on the Training set

In [24]:
# default batch size is 32, a neural netwrok has be trained on a certain number of epochs
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.5318 - accuracy: 0.7958
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4702 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4468 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4330 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4258 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4215 - accuracy: 0.8008
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4181 - accuracy: 0.8154
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4158 - accuracy: 0.8241
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4128 - accuracy: 0.8284
Epoch 10/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4105 - accura

250/250 [==============================] - 1s 4ms/step - loss: 0.3366 - accuracy: 0.8636
Epoch 82/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3369 - accuracy: 0.8634
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3362 - accuracy: 0.8644
Epoch 84/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3364 - accuracy: 0.8634
Epoch 85/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3364 - accuracy: 0.8636
Epoch 86/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3360 - accuracy: 0.8640
Epoch 87/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3362 - accuracy: 0.8639
Epoch 88/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3362 - accuracy: 0.8644
Epoch 89/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3361 - accuracy: 0.8646
Epoch 90/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3360 - accuracy: 

Homework

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [26]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 167ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

# Predicting the Test set results

In [28]:
Y_pred = ann.predict(X_test)
Y_pred = (Y_pred > 0.5) # this will convert the result as 0 or 1
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Making the Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
accuracy_score(Y_test, Y_pred)

[[1498   97]
 [ 187  218]]


0.858

In [33]:
# 1498 - correct prediction that customer stays in the bank
# 218 - correct prediction that customer leaves the bank

# 97 - incorrect prediction that customer leaves the bank
# 187 - incorrect prediction that customer stays in the bank